The training process is to train a neural network in a manner of supervised learning process using a convolutional neural network architecture. 

The network classifies whether the url string is beningn or malicious based on the url String.

In [36]:
# Package Imports
import pandas as pd
import numpy as np
from string import printable
from sklearn import model_selection
from keras.preprocessing import sequence
from keras import regularizers
from keras.layers import Input, Embedding

In [5]:
# Importing dataset
dataset = pd.read_csv('./dataset/urlString.csv')

In [6]:
dataset.head(20)

,url,isMalicious
0,songlyrics.com/news/riffd-the-shins-heartworms,0
1,imaging-resource.com/PRODS/olympus-e-m1-ii/oly...,0
2,gosugamers.net/lol/streams,0
3,thingiverse.com/corkyzett/collections/intlwome...,0
4,bausch.com/our-products/contact-lens-care/spec...,0
5,w88mobile.org/game/5151.html,0
6,datacenterdynamics.com/awards/latin-america-aw...,0
7,123people.com/s/marc+pageau,0
8,nownews.com/n/2017/03/21/2449327,0
9,wikipedia.org/wiki/Plague_Park,0


In [7]:
len(dataset)

194798

In [10]:
# List of printable characters
printable

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [14]:
# dataset preparation
# encode the printable characters in the url string as integers
def urlPrep(url):
    return [printable.index(char) + 1 for char in url if char in printable]

In [15]:
url_tokens = []
for url in dataset.url:
    url_tokens.append(urlPrep(url))

In [16]:
url_tokens

[[29,
  25,
  24,
  17,
  22,
  35,
  28,
  19,
  13,
  29,
  76,
  13,
  25,
  23,
  77,
  24,
  15,
  33,
  29,
  77,
  28,
  19,
  16,
  16,
  14,
  75,
  30,
  18,
  15,
  75,
  29,
  18,
  19,
  24,
  29,
  75,
  18,
  15,
  11,
  28,
  30,
  33,
  25,
  28,
  23,
  29],
 [19,
  23,
  11,
  17,
  19,
  24,
  17,
  75,
  28,
  15,
  29,
  25,
  31,
  28,
  13,
  15,
  76,
  13,
  25,
  23,
  77,
  52,
  54,
  51,
  40,
  55,
  77,
  25,
  22,
  35,
  23,
  26,
  31,
  29,
  75,
  15,
  75,
  23,
  2,
  75,
  19,
  19,
  77,
  25,
  22,
  35,
  23,
  26,
  31,
  29,
  75,
  15,
  75,
  23,
  2,
  75,
  19,
  19,
  37,
  76,
  44,
  56,
  49],
 [17,
  25,
  29,
  31,
  17,
  11,
  23,
  15,
  28,
  29,
  76,
  24,
  15,
  30,
  77,
  22,
  25,
  22,
  77,
  29,
  30,
  28,
  15,
  11,
  23,
  29],
 [30,
  18,
  19,
  24,
  17,
  19,
  32,
  15,
  28,
  29,
  15,
  76,
  13,
  25,
  23,
  77,
  13,
  25,
  28,
  21,
  35,
  36,
  15,
  30,
  30,
  77,
  13,
  25,
  22,
  22,
  15,
  1

In [21]:
# Standardize the url string length in the dataset taking only the first 75 printable characters or padding with zeros
max_length = 75
url_data = sequence.pad_sequences(url_tokens, maxlen=max_length)

In [23]:
# target_label array
target_label = np.array(dataset.isMalicious)

In [24]:
# Shape of the encoded url string array
url_data.shape

(194798, 75)

In [25]:
# Shape of the target labels array
target_label.shape

(194798,)

In [27]:
# splitting the url dataset using 25 percent for testing
url_train, url_test, target_train, target_test = model_selection.train_test_split(url_data, target_label, test_size=0.25, random_state=30)